In [1]:
!pip install -U gensim==4.0.0
!pip install python-Levenshtein
!pip install textattack

     |████████████████████████████████| 23.9MB 130kB/s 
  Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0
     |████████████████████████████████| 51kB 4.5MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149803 sha256=82fc57cc10f854445d515044b56c9e36ef485a12e68c24db952f28433459e894
  Stored in directory: /root/.cache/pip/wheels/b3/26/73/4b48503bac73f01cf18e52cd250947049a7f339e940c5df8fc
Successfully built python-Levenshtein
     |████████████████████████████████| 358kB 10.7MB/s 
     |████████████████████████████████| 778kB 20.3MB/s 
     |████████████████████████████████| 102kB 10.3MB/s 
     |████████████████████████████████| 337kB 35.4MB/s 
     |████████████████████████████████| 20.1MB 1.3MB/s 
     |████████████████████████████████| 2.1MB 38.1MB/s 
     |████████████████████████████████| 225kB 43.2MB/s 
     |████████████████████████████████| 61kB 7.

In [11]:
from collections import deque

import textattack
import tqdm
import time
from IPython.display import display, HTML

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from gensim.test.utils import get_tmpfile
from gensim.models.fasttext import FastText
#fname = get_tmpfile("/models/fasttext.model") 
model1 = FastText.load("./drive/MyDrive/models/fasttext.model") # Change this location

In [4]:
from textattack.transformations import WordSwap

class Swapper(WordSwap): # For fasttext
    """ Transforms an input by replacing any word with its most similar counterpart
    """

    # We don't need a constructor, since our class doesn't require any parameters.

    def _get_replacement_words(self, word):
        for i in range(10):
            if word.lower() in model1.wv.most_similar(word)[i][0].lower():
                continue # Don't return a word containing the exact word
            elif word.isupper():
                return [model1.wv.most_similar(word)[i][0].upper()] # Preserving case
            elif word[0].isupper():
                return [model1.wv.most_similar(word)[i][0].capitalize()] # Preserving Capitalization in words
            else:
                return [model1.wv.most_similar(word)[i][0]]
            
        return [model1.wv.most_similar(word)[0][0]]

textattack: Updating TextAttack package dependencies.
textattack: Downloading NLTK required packages.


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw.zip.
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


textattack: Downloading https://textattack.s3.amazonaws.com/word_embeddings/paragramcf.
100%|██████████| 481M/481M [00:17<00:00, 28.1MB/s]
textattack: Unzipping file /root/.cache/textattack/tmpgyestdk0.zip to /root/.cache/textattack/word_embeddings/paragramcf.
textattack: Successfully saved word_embeddings/paragramcf to cache.


In [6]:
# Import the model
import transformers
from textattack.models.tokenizers import AutoTokenizer
from textattack.models.wrappers import HuggingFaceModelWrapper
from textattack.datasets import HuggingFaceDataset

model = transformers.AutoModelForSequenceClassification.from_pretrained("textattack/albert-base-v2-yelp-polarity")
tokenizer = AutoTokenizer("textattack/albert-base-v2-yelp-polarity")
# Change these

model_wrapper = HuggingFaceModelWrapper(model, tokenizer)
dataset = HuggingFaceDataset("yelp_polarity", None, "test") # Change the dataset

textattack: Loading datasets dataset yelp_polarity, split test.


Dataset yelp_polarity downloaded and prepared to /root/.cache/huggingface/datasets/yelp_polarity/plain_text/1.0.0/591dc79cb354c6f051b5bf61c77a446205c98156e01669c6218c9bd6aa15f581. Subsequent calls will reuse this data.


# Stats of FastAttack

In [9]:
from textattack.constraints.grammaticality import PartOfSpeech
from textattack.constraints.pre_transformation import (
    InputColumnModification,
    RepeatModification,
    StopwordModification,
)
from textattack.constraints.overlap import MaxWordsPerturbed
from textattack.constraints.semantics import WordEmbeddingDistance
from textattack.constraints.semantics.sentence_encoders import UniversalSentenceEncoder
from textattack.search_methods import GreedyWordSwapWIR
from textattack.constraints.pre_transformation import RepeatModification, StopwordModification
from textattack.shared import Attack
from textattack.goal_functions import UntargetedClassification
from textattack.datasets import HuggingFaceDataset

# We're going to use our word swap class as the attack transformation.
transformation = Swapper()
# We'll constrain modification of already modified indices and stopwords
constraints = [RepeatModification(), StopwordModification()]
# # We'll use the Greedy search method
search_method = GreedyWordSwapWIR("weighted-saliency")
# Create the goal function using the model

goal_function = UntargetedClassification(model_wrapper)

# Now, let's make the attack from the 4 components:
attack = Attack(goal_function, constraints, transformation, search_method)

textattack: Unknown if model of class <class 'transformers.models.albert.modeling_albert.AlbertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [7]:
def attack_stats(attack, dataset, num_of_examples):
  print(attack)
  num_examples = num_of_examples
  num_remaining_attacks = num_examples
  pbar = tqdm.tqdm(total=num_remaining_attacks, smoothing=0)

  worklist = deque(range(0, num_examples))
  worklist_tail = worklist[-1]

  attack_log_manager = textattack.loggers.AttackLogManager()

  load_time = time.time()
        
  num_results = 0
  num_failures = 0
  num_successes = 0
  for result in attack.attack_dataset(dataset, indices=worklist):
      result_html_str = result.__str__(color_method="html").replace("\n\n", "<br>")
      display(HTML(result_html_str))
      attack_log_manager.log_result(result)
            
      if not isinstance(result, textattack.attack_results.SkippedAttackResult):
          pbar.update(1)
      else:
          worklist_tail += 1
          pbar.update(1)
          worklist.append(worklist_tail)

      num_results += 1

      if (type(result) == textattack.attack_results.SuccessfulAttackResult
        or type(result) == textattack.attack_results.MaximizedAttackResult):
          num_successes += 1
                
      if type(result) == textattack.attack_results.FailedAttackResult:
          num_failures += 1
      pbar.set_description(
          "[Succeeded / Failed / Total] {} / {} / {}".format(
            num_successes, num_failures, num_results
        )
      )

  pbar.close()

  attack_log_manager.enable_stdout()
  attack_log_manager.log_summary()
  attack_log_manager.flush()
        
  textattack.shared.logger.info(f"Attack time: {time.time() - load_time}s")

In [12]:
attack_stats(attack, dataset, 5)

  0%|          | 0/5 [00:00<?, ?it/s]

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  weighted-saliency
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


[Succeeded / Failed / Total] 1 / 0 / 1:  20%|██        | 1/5 [00:09<00:36,  9.17s/it]

[Succeeded / Failed / Total] 2 / 0 / 2:  40%|████      | 2/5 [00:15<00:23,  7.70s/it]

[Succeeded / Failed / Total] 3 / 0 / 3:  60%|██████    | 3/5 [00:16<00:11,  5.54s/it]

[Succeeded / Failed / Total] 4 / 0 / 4:  80%|████████  | 4/5 [00:20<00:05,  5.04s/it]

[Succeeded / Failed / Total] 4 / 1 / 5: 100%|██████████| 5/5 [00:35<00:00,  7.11s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 4      |
| Number of failed attacks:     | 1      |
| Number of skipped attacks:    | 0      |
| Original accuracy:            | 100.0% |
| Accuracy under attack:        | 20.0%  |
| Attack success rate:          | 80.0%  |
| Average perturbed word %:     | 9.74%  |
| Average num. words per input: | 80.4   |
| Avg num queries:              | 153.2  |
+-------------------------------+--------+


textattack: Attack time: 35.546950340270996s


In [13]:
attack_stats(attack, dataset, 10)

  0%|          | 0/10 [00:00<?, ?it/s]

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  weighted-saliency
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)


[Succeeded / Failed / Total] 1 / 0 / 1:  10%|█         | 1/10 [00:08<01:19,  8.83s/it]

[Succeeded / Failed / Total] 2 / 0 / 2:  20%|██        | 2/10 [00:15<01:00,  7.53s/it]

[Succeeded / Failed / Total] 3 / 0 / 3:  30%|███       | 3/10 [00:16<00:38,  5.43s/it]

...

[Succeeded / Failed / Total] 6 / 2 / 8:  80%|████████  | 8/10 [01:22<00:20, 10.26s/it]

[Succeeded / Failed / Total] 7 / 2 / 9:  90%|█████████ | 9/10 [01:36<00:10, 10.73s/it]

[Succeeded / Failed / Total] 8 / 2 / 10: 100%|██████████| 10/10 [01:42<00:00, 10.20s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 8      |
| Number of failed attacks:     | 2      |
| Number of skipped attacks:    | 0      |
| Original accuracy:            | 100.0% |
| Accuracy under attack:        | 20.0%  |
| Attack success rate:          | 80.0%  |
| Average perturbed word %:     | 9.37%  |
| Average num. words per input: | 121.2  |
| Avg num queries:              | 219.9  |
+-------------------------------+--------+


textattack: Attack time: 102.04814577102661s


In [15]:
attack_stats(attack, dataset, 15)


  0%|          | 0/15 [00:00<?, ?it/s]

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  weighted-saliency
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)



[Succeeded / Failed / Total] 1 / 0 / 1:   7%|▋         | 1/15 [00:09<02:06,  9.06s/it]


[Succeeded / Failed / Total] 2 / 0 / 2:  13%|█▎        | 2/15 [00:15<01:39,  7.67s/it]


[Succeeded / Failed / Total] 3 / 0 / 3:  20%|██        | 3/15 [00:16<01:06,  5.53s/it]

...


[Succeeded / Failed / Total] 10 / 3 / 13:  87%|████████▋ | 13/15 [02:33<00:23, 11.78s/it]


[Succeeded / Failed / Total] 11 / 3 / 14:  93%|█████████▎| 14/15 [02:43<00:11, 11.69s/it]


[Succeeded / Failed / Total] 12 / 3 / 15: 100%|██████████| 15/15 [02:47<00:00, 11.19s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 12     |
| Number of failed attacks:     | 3      |
| Number of skipped attacks:    | 0      |
| Original accuracy:            | 100.0% |
| Accuracy under attack:        | 20.0%  |
| Attack success rate:          | 80.0%  |
| Average perturbed word %:     | 11.31% |
| Average num. words per input: | 130.07 |
| Avg num queries:              | 239.73 |
+-------------------------------+--------+


textattack: Attack time: 167.8620867729187s


In [16]:
attack_stats(attack, dataset, 20)


  0%|          | 0/20 [00:00<?, ?it/s]

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  weighted-saliency
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)



[Succeeded / Failed / Total] 1 / 0 / 1:   5%|▌         | 1/20 [00:08<02:48,  8.88s/it]


[Succeeded / Failed / Total] 2 / 0 / 2:  10%|█         | 2/20 [00:15<02:15,  7.54s/it]


[Succeeded / Failed / Total] 3 / 0 / 3:  15%|█▌        | 3/20 [00:16<01:32,  5.44s/it]

...


[Succeeded / Failed / Total] 15 / 3 / 18:  90%|█████████ | 18/20 [03:30<00:23, 11.69s/it]


[Succeeded / Failed / Total] 15 / 4 / 19:  95%|█████████▌| 19/20 [03:52<00:12, 12.26s/it]


[Succeeded / Failed / Total] 15 / 5 / 20: 100%|██████████| 20/20 [04:01<00:00, 12.08s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 15     |
| Number of failed attacks:     | 5      |
| Number of skipped attacks:    | 0      |
| Original accuracy:            | 100.0% |
| Accuracy under attack:        | 25.0%  |
| Attack success rate:          | 75.0%  |
| Average perturbed word %:     | 10.2%  |
| Average num. words per input: | 140.0  |
| Avg num queries:              | 256.6  |
+-------------------------------+--------+


textattack: Attack time: 241.53275442123413s


In [17]:
attack_stats(attack, dataset, 25)


  0%|          | 0/25 [00:00<?, ?it/s]

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  weighted-saliency
  )
  (goal_function):  UntargetedClassification
  (transformation):  Swapper
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
)



[Succeeded / Failed / Total] 1 / 0 / 1:   4%|▍         | 1/25 [00:08<03:32,  8.85s/it]


[Succeeded / Failed / Total] 2 / 0 / 2:   8%|▊         | 2/25 [00:15<02:53,  7.53s/it]


[Succeeded / Failed / Total] 3 / 0 / 3:  12%|█▏        | 3/25 [00:16<01:59,  5.43s/it]

...


[Succeeded / Failed / Total] 18 / 5 / 23:  92%|█████████▏| 23/25 [04:25<00:23, 11.53s/it]


[Succeeded / Failed / Total] 19 / 5 / 24:  96%|█████████▌| 24/25 [04:40<00:11, 11.70s/it]


[Succeeded / Failed / Total] 19 / 6 / 25: 100%|██████████| 25/25 [04:44<00:00, 11.37s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 19     |
| Number of failed attacks:     | 6      |
| Number of skipped attacks:    | 0      |
| Original accuracy:            | 100.0% |
| Accuracy under attack:        | 24.0%  |
| Attack success rate:          | 76.0%  |
| Average perturbed word %:     | 11.43% |
| Average num. words per input: | 133.36 |
| Avg num queries:              | 242.76 |
+-------------------------------+--------+


textattack: Attack time: 284.3558597564697s


# TextFooler's stats

In [18]:
!pip install torchfile
from textattack.constraints.grammaticality import PartOfSpeech
from textattack.constraints.pre_transformation import (
    InputColumnModification,
    RepeatModification,
    StopwordModification,
)
from textattack.constraints.semantics import WordEmbeddingDistance
from textattack.constraints.semantics.sentence_encoders import UniversalSentenceEncoder
from textattack.goal_functions import UntargetedClassification
from textattack.search_methods import GreedyWordSwapWIR
from textattack.shared.attack import Attack
from textattack.transformations import WordSwapEmbedding

from textattack.shared import Attack
from textattack.goal_functions import UntargetedClassification
from textattack.datasets import HuggingFaceDataset

transformation = WordSwapEmbedding(max_candidates=50)
        #
        # Don't modify the same word twice or the stopwords defined
        # in the TextFooler public implementation.
        #
        # fmt: off
stopwords = set(
            ["a", "about", "above", "across", "after", "afterwards", "again", "against", "ain", "all", "almost", "alone", "along", "already", "also", "although", "am", "among", "amongst", "an", "and", "another", "any", "anyhow", "anyone", "anything", "anyway", "anywhere", "are", "aren", "aren't", "around", "as", "at", "back", "been", "before", "beforehand", "behind", "being", "below", "beside", "besides", "between", "beyond", "both", "but", "by", "can", "cannot", "could", "couldn", "couldn't", "d", "didn", "didn't", "doesn", "doesn't", "don", "don't", "down", "due", "during", "either", "else", "elsewhere", "empty", "enough", "even", "ever", "everyone", "everything", "everywhere", "except", "first", "for", "former", "formerly", "from", "hadn", "hadn't", "hasn", "hasn't", "haven", "haven't", "he", "hence", "her", "here", "hereafter", "hereby", "herein", "hereupon", "hers", "herself", "him", "himself", "his", "how", "however", "hundred", "i", "if", "in", "indeed", "into", "is", "isn", "isn't", "it", "it's", "its", "itself", "just", "latter", "latterly", "least", "ll", "may", "me", "meanwhile", "mightn", "mightn't", "mine", "more", "moreover", "most", "mostly", "must", "mustn", "mustn't", "my", "myself", "namely", "needn", "needn't", "neither", "never", "nevertheless", "next", "no", "nobody", "none", "noone", "nor", "not", "nothing", "now", "nowhere", "o", "of", "off", "on", "once", "one", "only", "onto", "or", "other", "others", "otherwise", "our", "ours", "ourselves", "out", "over", "per", "please", "s", "same", "shan", "shan't", "she", "she's", "should've", "shouldn", "shouldn't", "somehow", "something", "sometime", "somewhere", "such", "t", "than", "that", "that'll", "the", "their", "theirs", "them", "themselves", "then", "thence", "there", "thereafter", "thereby", "therefore", "therein", "thereupon", "these", "they", "this", "those", "through", "throughout", "thru", "thus", "to", "too", "toward", "towards", "under", "unless", "until", "up", "upon", "used", "ve", "was", "wasn", "wasn't", "we", "were", "weren", "weren't", "what", "whatever", "when", "whence", "whenever", "where", "whereafter", "whereas", "whereby", "wherein", "whereupon", "wherever", "whether", "which", "while", "whither", "who", "whoever", "whole", "whom", "whose", "why", "with", "within", "without", "won", "won't", "would", "wouldn", "wouldn't", "y", "yet", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves"]
        )
        # fmt: on
constraints = [RepeatModification(), StopwordModification(stopwords=stopwords)]
        #
        # During entailment, we should only edit the hypothesis - keep the premise
        # the same.
        #
input_column_modification = InputColumnModification(
            ["premise", "hypothesis"], {"premise"}
        )
constraints.append(input_column_modification)
        # Minimum word embedding cosine similarity of 0.5.
        # (The paper claims 0.7, but analysis of the released code and some empirical
        # results show that it's 0.5.)
        #
constraints.append(WordEmbeddingDistance(min_cos_sim=0.5))
        #
        # Only replace words with the same part of speech (or nouns with verbs)
        #
constraints.append(PartOfSpeech(allow_verb_noun_swap=True))
        #
        # Universal Sentence Encoder with a minimum angular similarity of ε = 0.5.
        #
        # In the TextFooler code, they forget to divide the angle between the two
        # embeddings by pi. So if the original threshold was that 1 - sim >= 0.5, the
        # new threshold is 1 - (0.5) / pi = 0.840845057
        #
use_constraint = UniversalSentenceEncoder(
            threshold=0.840845057,
            metric="angular",
            compare_against_original=False,
            window_size=15,
            skip_text_shorter_than_window=True,
        )
constraints.append(use_constraint)
        #
        # Goal is untargeted classification
        #
goal_function = UntargetedClassification(model_wrapper)
        #
        # Greedily swap words with "Word Importance Ranking".
        #
search_method = GreedyWordSwapWIR(wir_method="delete")

# Now, let's make the attack from the 4 components:
attack1 = Attack(goal_function, constraints, transformation, search_method)

  Created wheel for torchfile: filename=torchfile-0.1.0-cp37-none-any.whl size=5713 sha256=0024b802627676ab9347982892917fec1d85899e616a6551574bcf8967108595
  Stored in directory: /root/.cache/pip/wheels/b1/c3/d6/9a1cc8f3a99a0fc1124cae20153f36af59a6e683daca0a0814
Successfully built torchfile


Using /tmp/tfhub_modules to cache modules.
Downloaded https://tfhub.dev/google/universal-sentence-encoder/4, Total size: 987.47MB
Downloaded TF-Hub Module 'https://tfhub.dev/google/universal-sentence-encoder/4'.
textattack: Unknown if model of class <class 'transformers.models.albert.modeling_albert.AlbertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [19]:
attack_stats(attack1, dataset, 5)


  0%|          | 0/5 [00:00<?, ?it/s]

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       


[Succeeded / Failed / Total] 1 / 0 / 1:  20%|██        | 1/5 [00:12<00:50, 12.75s/it]


[Succeeded / Failed / Total] 2 / 0 / 2:  40%|████      | 2/5 [00:20<00:30, 10.10s/it]


[Succeeded / Failed / Total] 3 / 0 / 3:  60%|██████    | 3/5 [00:21<00:14,  7.25s/it]


[Succeeded / Failed / Total] 4 / 0 / 4:  80%|████████  | 4/5 [00:25<00:06,  6.33s/it]


[Succeeded / Failed / Total] 5 / 0 / 5: 100%|██████████| 5/5 [00:37<00:00,  7.52s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 5      |
| Number of failed attacks:     | 0      |
| Number of skipped attacks:    | 0      |
| Original accuracy:            | 100.0% |
| Accuracy under attack:        | 0.0%   |
| Attack success rate:          | 100.0% |
| Average perturbed word %:     | 5.6%   |
| Average num. words per input: | 80.4   |
| Avg num queries:              | 192.0  |
+-------------------------------+--------+


textattack: Attack time: 37.602774143218994s


In [20]:
attack_stats(attack1, dataset, 10)


  0%|          | 0/10 [00:00<?, ?it/s]

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       


[Succeeded / Failed / Total] 1 / 0 / 1:  10%|█         | 1/10 [00:09<01:27,  9.75s/it]


[Succeeded / Failed / Total] 2 / 0 / 2:  20%|██        | 2/10 [00:17<01:08,  8.59s/it]


[Succeeded / Failed / Total] 3 / 0 / 3:  30%|███       | 3/10 [00:18<00:43,  6.25s/it]

...


[Succeeded / Failed / Total] 7 / 1 / 8:  80%|████████  | 8/10 [01:30<00:22, 11.31s/it]


[Succeeded / Failed / Total] 8 / 1 / 9:  90%|█████████ | 9/10 [01:44<00:11, 11.62s/it]


[Succeeded / Failed / Total] 9 / 1 / 10: 100%|██████████| 10/10 [01:53<00:00, 11.34s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 9      |
| Number of failed attacks:     | 1      |
| Number of skipped attacks:    | 0      |
| Original accuracy:            | 100.0% |
| Accuracy under attack:        | 10.0%  |
| Attack success rate:          | 90.0%  |
| Average perturbed word %:     | 5.61%  |
| Average num. words per input: | 121.2  |
| Avg num queries:              | 313.5  |
+-------------------------------+--------+


textattack: Attack time: 113.43649959564209s


In [21]:
attack_stats(attack1, dataset, 15)


  0%|          | 0/15 [00:00<?, ?it/s]

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       


[Succeeded / Failed / Total] 1 / 0 / 1:   7%|▋         | 1/15 [00:09<02:17,  9.79s/it]


[Succeeded / Failed / Total] 2 / 0 / 2:  13%|█▎        | 2/15 [00:17<01:52,  8.62s/it]


[Succeeded / Failed / Total] 3 / 0 / 3:  20%|██        | 3/15 [00:18<01:15,  6.27s/it]

...


[Succeeded / Failed / Total] 12 / 1 / 13:  87%|████████▋ | 13/15 [03:59<00:36, 18.41s/it]


[Succeeded / Failed / Total] 13 / 1 / 14:  93%|█████████▎| 14/15 [04:08<00:17, 17.77s/it]


[Succeeded / Failed / Total] 14 / 1 / 15: 100%|██████████| 15/15 [04:12<00:00, 16.86s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 14     |
| Number of failed attacks:     | 1      |
| Number of skipped attacks:    | 0      |
| Original accuracy:            | 100.0% |
| Accuracy under attack:        | 6.67%  |
| Attack success rate:          | 93.33% |
| Average perturbed word %:     | 7.95%  |
| Average num. words per input: | 130.07 |
| Avg num queries:              | 452.2  |
+-------------------------------+--------+


textattack: Attack time: 252.93022108078003s


In [22]:
attack_stats(attack1, dataset, 20)


  0%|          | 0/20 [00:00<?, ?it/s]

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       


[Succeeded / Failed / Total] 1 / 0 / 1:   5%|▌         | 1/20 [00:09<03:04,  9.71s/it]


[Succeeded / Failed / Total] 2 / 0 / 2:  10%|█         | 2/20 [00:17<02:34,  8.56s/it]


[Succeeded / Failed / Total] 3 / 0 / 3:  15%|█▌        | 3/20 [00:18<01:45,  6.22s/it]

...


[Succeeded / Failed / Total] 17 / 1 / 18:  90%|█████████ | 18/20 [04:45<00:31, 15.86s/it]


[Succeeded / Failed / Total] 18 / 1 / 19:  95%|█████████▌| 19/20 [06:12<00:19, 19.62s/it]


[Succeeded / Failed / Total] 19 / 1 / 20: 100%|██████████| 20/20 [06:18<00:00, 18.91s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 19     |
| Number of failed attacks:     | 1      |
| Number of skipped attacks:    | 0      |
| Original accuracy:            | 100.0% |
| Accuracy under attack:        | 5.0%   |
| Attack success rate:          | 95.0%  |
| Average perturbed word %:     | 8.56%  |
| Average num. words per input: | 140.0  |
| Avg num queries:              | 502.8  |
+-------------------------------+--------+


textattack: Attack time: 378.2717475891113s


In [23]:
attack_stats(attack1, dataset, 25)


  0%|          | 0/25 [00:00<?, ?it/s]

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       


[Succeeded / Failed / Total] 1 / 0 / 1:   4%|▍         | 1/25 [00:09<03:52,  9.70s/it]


[Succeeded / Failed / Total] 2 / 0 / 2:   8%|▊         | 2/25 [00:17<03:16,  8.54s/it]


[Succeeded / Failed / Total] 3 / 0 / 3:  12%|█▏        | 3/25 [00:18<02:16,  6.21s/it]

...


[Succeeded / Failed / Total] 22 / 1 / 23:  92%|█████████▏| 23/25 [06:53<00:35, 17.99s/it]


[Succeeded / Failed / Total] 23 / 1 / 24:  96%|█████████▌| 24/25 [07:25<00:18, 18.58s/it]


[Succeeded / Failed / Total] 24 / 1 / 25: 100%|██████████| 25/25 [07:30<00:00, 18.01s/it]

+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 24     |
| Number of failed attacks:     | 1      |
| Number of skipped attacks:    | 0      |
| Original accuracy:            | 100.0% |
| Accuracy under attack:        | 4.0%   |
| Attack success rate:          | 96.0%  |
| Average perturbed word %:     | 9.09%  |
| Average num. words per input: | 133.36 |
| Avg num queries:              | 480.92 |
+-------------------------------+--------+


textattack: Attack time: 450.33698773384094s
